In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm

from gensim.models import KeyedVectors


In [2]:
dmeeting = pd.read_parquet('./data/data_meeting.parquet')
data_agenda1 = pd.read_parquet('./data/data_agenda1.parquet')
data_agenda2 = pd.read_parquet('./data/data_agenda2.parquet')
data_agenda3 = pd.read_parquet('./data/data_agenda3.parquet')
data_speech1 = pd.read_parquet('./data/data_speech1.parquet')
data_speech2 = pd.read_parquet('./data/data_speech2.parquet')
data_speech3 = pd.read_parquet('./data/data_speech3.parquet')
parMem = pd.read_parquet('./data/parliament_members.parquet')

dagenda = pd.concat([data_agenda1, data_agenda2, data_agenda3], axis=0)
dspeech = pd.concat([data_speech1, data_speech2, data_speech3], axis=0)

annotation_data = pd.read_csv('data/annotation_data.csv')

In [3]:
annotation_data["id"] = annotation_data["meeting_id"].astype(str) + "_" + annotation_data["agenda_item_id"].astype(str)
dspeech["id"] = dspeech["meeting_id"].astype(str) + "_" + dspeech["agenda_item_id"].astype(str)
dspeech = dspeech.loc[~dspeech.id.isin(annotation_data.id)].reset_index(drop=False).reset_index(drop=True)

In [10]:

MODEL_FILE = '../../dsl_skipgram_2020_m5_f500_epoch2_w5.model.w2v.bin'
model = KeyedVectors.load_word2vec_format(MODEL_FILE, binary=True)

In [31]:
exception_list = ['CO2', 'co2']
nlp = spacy.load("da_core_news_sm")

def preprocess_text(text):
    doc = nlp(text)
    processed_tokens = []
    vec_representation = np.zeros((500))
    for token in doc:
        print(token, token.is_stop)
        if token.is_stop:
            continue
        elif not token.is_alpha:
            continue
        elif token in exception_list:
            continue
        token = token.text.lower()
        processed_tokens.append(token)
        vec = model.word_vec(token)
        vec_representation += vec/len(doc)
    # processed_tokens = [token.text.lower()for token in doc if not token.is_stop and token.is_alpha or token.text in exception_list]
    return processed_tokens, vec_representation

In [32]:
tokenized_text= []
s_i_embeddings = []
for speech_item in tqdm(dspeech["speech_item_text"].to_list()[0:5]):
    tokenized, s_i_e = preprocess_text(speech_item)
    tokenized_text.append(tokenized)
    s_i_embeddings.append(s_i_e)
    
print(tokenized_text[0])
print(s_i_embeddings[0])
#dspeech["tokenized_text"] = tokenized_text
#dspeech["s_i_embeddings"] = s_i_embeddings
dspeech.to_parquet('data/data_speech_tokenized.parquet') 

  0%|          | 0/5 [00:00<?, ?it/s]/var/folders/90/hzvrpycd5ds6jx99ygz9bbk00000gn/T/ipykernel_58523/1587164108.py:16: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  vec = model.word_vec(token)
100%|██████████| 5/5 [00:00<00:00, 56.14it/s]


Tak False
. False
Den True
tidligere True
transportminister False
var True
optaget False
af True
, False
at True
vi True
fik False
sikkerhed False
for True
, False
at True
togene False
kunne True
bruges False
på True
en True
måde False
, False
så True
folk False
også True
havde True
mulighed False
for True
at True
tage False
med True
dem True
. False
I True
den True
forbindelse False
havde True
vi True
så True
spørgsmålet False
oppe False
om True
stationsåbninger False
. False
Nu True
er True
det True
sådan True
, False
at True
på True
Fyn False
er True
der True
stationsåbninger False
vest False
for True
Odense False
, False
men True
der True
er True
slet False
ikke True
nogen True
øst False
for True
Odense False
. False
Der True
har True
vi True
tidligere True
talt False
om True
muligheden False
for True
at True
få True
åbnet False
Langeskov False
Station False
, False
for True
der True
ligger False
færdige False
stationsbygninger False
, False
og True
der True
er True
plads False
til

In [27]:
dspeech_sample["tokenized_text"] = tokenized_text

In [145]:
with open('data/C_word_list.txt', 'r', encoding='utf-8') as f:
    C_words_list = f.read().splitlines()
    C_words_set = set(C_words_list)

with open('data/NC_word_list.txt', 'r', encoding='utf-8') as f:
    NC_words_list = f.read().splitlines()
    NC_words_set = set(NC_words_list)

with open('data/NC_word_list_all.txt', 'r', encoding='utf-8') as f:
    NC_words_all = f.read().splitlines()
    NC_words_set_all = set(NC_words_all)


In [146]:
label_counts = []
for speech_item in tqdm(dspeech_sample['tokenized_text'].to_list()):
    C_count = 0
    NC_count = 0
    NC_count_all = 0
    for word in speech_item:
        if word in C_words_set:
            C_count += 1
        elif word in NC_words_set:
            NC_count += 1
        elif word in NC_words_set_all:
            NC_count_all += 1
    label_counts.append([C_count, NC_count, NC_count_all])

dspeech_sample[['C_counts', 'NC_counts', 'NC_counts_all']] = label_counts


100%|██████████| 33207/33207 [00:00<00:00, 99239.71it/s] 


In [136]:
#dspeech_sample.to_csv('data_additionaal/tokenized33000.csv')

In [147]:
dspeech_sample.head(10)

,level_0,index,meeting_id,agenda_item_id,speech_item_id,time_start,time_end,speaker_name,speaker_party,speaker_role,speaker_title,speech_item_text,duration,number_of_words,id,tokenized_text,C_counts,NC_counts,NC_counts_all
42,42,42,1,8,3,13:47:24,13:49:43,Tina Nedergaard,V,minister,Undervisningsministeren(Tina Nedergaard):,"Jeg vil bare slå fast, at hele vores voksen- o...",139.0,510,1_V,"[bare, slå, fast, hele, vigtigt, klart, hele, ...",2,1,3
48,48,48,1,9,3,13:55:57,13:57:54,Tina Nedergaard,V,minister,Undervisningsministeren(Tina Nedergaard):,Jeg ville i sådan en situation helt klart anbe...,117.0,426,1_V,"[situation, helt, klart, anbefale, ventede, år...",2,0,13
74,74,74,1,12,12,14:28:49,14:29:12,Henrik Høegh,V,minister,Fødevareministeren(Henrik Høegh):,"Jeg kan ikke i dag svare på, hvordan Kommissio...",23.0,58,1_V,"[dag, svare, kommissionen, reagere, enige, par...",0,0,0
95,95,103,1,17,6,15:03:43,15:05:05,Ole Vagn Christensen,S,medlem,Ole Vagn Christensen(S):,"Jeg kunne forstå, at det, som beskæftigelsesmi...",82.0,125,1_S,"[forstå, beskæftigelsesministeren, sagde, find...",0,0,6
116,116,132,1,20,14,15:32:09,15:33:03,Thomas Jensen,S,medlem,Thomas Jensen(S):,Ministeren formåede jo næsten at svare fuldstæ...,54.0,193,1_S,"[ministeren, formåede, svare, fuldstændig, ude...",0,1,2
133,133,149,1,30,16,15:54:55,15:55:58,Karsten Hønge,SF,medlem,Karsten Hønge(SF):,"Ministeren bekræfter altså en tro på, at man k...",63.0,233,1_SF,"[ministeren, bekræfter, altså, tro, spare, ned...",0,4,7
138,138,154,1,31,8,16:01:51,16:02:56,Brian Mikkelsen,KF,minister,Økonomi- og erhvervsministeren(Brian Mikkelsen):,"Regeringen har jo et væld af aktiviteter, en h...",65.0,198,1_KF,"[regeringen, væld, aktiviteter, palet, initiat...",0,1,7
141,141,157,1,31,14,16:04:56,16:05:47,Karsten Hønge,SF,medlem,Karsten Hønge(SF):,Med en passiv besparelsesstrategi opnår man jo...,51.0,154,1_SF,"[passiv, besparelsesstrategi, opnår, kortsigte...",0,1,4
147,147,163,1,32,10,16:12:31,16:13:39,Høgni Hoydal,TF,medlem,Høgni Hoydal(TF):,"Jeg må sige, at det jo ikke er noget, hr. Høgn...",68.0,177,1_TF,"[høgni, hoydal, rejser, refererer, spørgsmålet...",0,1,1
155,155,171,1,33,10,16:22:14,16:23:23,Astrid Krag,SF,medlem,Astrid Krag(SF):,Vi kommer ikke så langt med at diskutere tekni...,69.0,206,1_SF,"[langt, diskutere, teknik, høre, diskutere, ho...",0,0,3


In [197]:
qsi = dspeech_sample[(dspeech_sample['C_counts']>10) & (dspeech_sample['NC_counts_all']< dspeech_sample['C_counts'])]

In [193]:
# those_eighty = dspeech_sample[(dspeech_sample['C_counts']>3) & (dspeech_sample['NC_counts']< dspeech_sample['C_counts']) & (dspeech_sample["NC_counts_all"]>= dspeech_sample['C_counts'])]

In [194]:
qsi.shape

(416, 19)

In [195]:
qsi['speaker_party'].value_counts()

speaker_party
V      72
S      72
EL     63
SF     50
RV     42
KF     28
DF     27
ALT    26
LA     21
NB      4
DD      4
M       4
FG      2
UFG     1
Name: count, dtype: int64

In [196]:
for text in qsi['speech_item_text'].to_list():
    print(text)
    print('-----------------')

Jeg vil starte med at kvittere for, at vi har fået det her lovforslag delt op, så vi får mulighed for at stemme imod den her del om partikelfiltre, og så lade regeringen fuldstændig selv stå med det ansvar. For det, vi har foran os, er et skatteforslag forklædt som et miljøforslag. Behandlingen af lovforslaget har været et dybt besynderligt forløb. Regeringen har ændret provenuberegninger, som vi andre skifter underbukser. Den har ændret forklaring om de tekniske muligheder for at eftermontere effektivt lukkede partikelfiltre. Vi har i det hele taget haft en større polemik og adskillige samråd og har altså fået lovforslaget delt, så jeg synes, det er på sin plads at sige noget her ved tredjebehandlingen. Sagen er jo, at luftforureningen i Danmarks store byer er massiv. Der er 3.400 mennesker, der årligt dør for tidligt på grund af luftforurening. Vi har 2 mio. sygedage hvert år i en tid, hvor vi også gør noget for at skaffe mere arbejdskraft, på grund af partikelforureningen. Regeringe